# Azure AutoML Local Compute Run 
## (Porto Seguro's Safe Driving Prediction)

Now let's use Azure Automated ML!

In this notebook you are going to use a **'Local Compute Run'** which is a better approach when getting started and trying small datasets because it needs less overhead on infrastructure preparation since it'll use your read-to-run Jupyter environment and compute from your machine (versus Azure ML remote runs which require to sping up VMs in the cloud for remote compute).

However, when targeting larger datasets and production trainings, it'll be better to use Azure ML Remote Compute since you will be able to parallelize runs/trainings and scale out on multiple VMs in the Azure ML training cluster. You can see that other approach in the 'automl-remote-compute-run-safe-driver-classifier.ipynb' notebook.

## Import Needed Packages

Import the packages needed for this solution notebook. The most widely used package for machine learning is [scikit-learn](https://scikit-learn.org/stable/), [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), and [numpy](https://numpy.org/). 

In [15]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn import metrics

## Check Azure ML SDK version

In [16]:
import azureml.core
print("This notebook was tested using version 1.6.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was tested using version 1.6.0 of the Azure ML SDK
You are currently using version 1.6.0 of the Azure ML SDK


##  Get Azure ML Workspace to use

In [17]:
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

## Load data from file into regular Pandas DataFrame

In [18]:
# Directly load from file into Pandas DataFrame
DATA_DIR = "../../data/"
data_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_train.csv'))

print(data_df.shape)
data_df.head(5)

(595212, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


## Split Data into Train and Test Sets

Partitioning data into training, validation, and holdout/test sets allows you to develop highly accurate models that are relevant to data that you collect in the future, not just the data the model was trained on. 

In this notebook we holdout a test dataset to calculate the metrics with that set "not seen" by AutoML, after the AutoML process is finished.
Not taking into account the test dataset, AutoML will by default internally either use a validation dataset split from the train dataset or use cross-validation, depending on the size of the data (larger than 20k rows will use validation split), or as explicitely specified in the AutoMLConfig class (Validation-split vs. Cross-Validation).

In [19]:
# Split in train/test datasets (Test=10%, Train=90%)

# Split in full sets, if not stratifying
train_df, test_df = train_test_split(data_df, test_size=0.1, random_state=0)

print(train_df.shape)
print(test_df.shape)

train_df.describe()

(535690, 59)
(59522, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,...,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00,535690.00
mean,743733.85,0.04,1.90,1.36,4.42,0.42,0.40,0.39,0.26,0.16,...,5.44,1.44,2.87,7.54,0.12,0.63,0.55,0.29,0.35,0.15
std,429511.16,0.19,1.98,0.67,2.70,0.49,1.35,0.49,0.44,0.37,...,2.33,1.20,1.69,2.75,0.33,0.48,0.50,0.45,0.48,0.36
min,7.00,0.00,0.00,-1.00,0.00,-1.00,-1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,371715.25,0.00,0.00,1.00,2.00,0.00,0.00,0.00,0.00,0.00,...,4.00,1.00,2.00,6.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,743327.00,0.00,1.00,1.00,4.00,0.00,0.00,0.00,0.00,0.00,...,5.00,1.00,3.00,7.00,0.00,1.00,1.00,0.00,0.00,0.00
75%,1115636.25,0.00,3.00,2.00,6.00,1.00,0.00,1.00,1.00,0.00,...,7.00,2.00,4.00,9.00,0.00,1.00,1.00,1.00,1.00,0.00
max,1488027.00,1.00,7.00,4.00,11.00,1.00,6.00,1.00,1.00,1.00,...,19.00,10.00,13.00,23.00,1.00,1.00,1.00,1.00,1.00,1.00


## Train with Azure AutoML automatically searching for the 'best model' (Best algorithms and best hyper-parameters)

### List and select primary metric to drive the AutoML classification problem

In [20]:
from azureml.train import automl
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric

['average_precision_score_weighted',
 'precision_score_weighted',
 'AUC_weighted',
 'accuracy',
 'norm_macro_recall']

## Define AutoML Experiment settings

In [24]:
import logging

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
automl_settings = {
      "blacklist_models":['LogisticRegression', 'ExtremeRandomTrees', 'RandomForest'], 
      # "whitelist_models": ['LightGBM'],
      "validation_size": 0.1,
      # "validation_data": validation_df,  # If you have an explicit validation set
      # "n_cross_validations": 5,
      # "experiment_exit_score": 0.7,
      # "max_cores_per_iteration": -1,
      "enable_voting_ensemble": True,
      "enable_stack_ensemble": True
}

from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             primary_metric='AUC_weighted',                           
                             training_data= train_df,  # Pandas DataFrame
                             label_column_name="target",                                                    
                             enable_early_stopping= True,
                             iterations=15,                    
                             experiment_timeout_hours=1,
                             featurization="auto",
                             model_explainability=True,
                             **automl_settings
                             )

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

## Run Experiment with multiple child runs under the covers

In [22]:
from azureml.core import Experiment

experiment_name = "SDK_local_porto_seguro_driver_pred"
print(experiment_name)

experiment = Experiment(workspace=ws, 
                        name=experiment_name)

import time
start_time = time.time()

run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s minutes needed for running the whole Remote AutoML Experiment ---' % ((time.time() - start_time)/60))

SDK_local_porto_seguro_driver_pred
Running on local machine
Parent Run ID: AutoML_51a190e0-5f8a-4742-b6ab-1db4153d3896

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------

## Explore results with Widget

In [23]:
# Explore the results of automatic training with a Jupyter widget: https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Retrieve the 'Best' Model

In [25]:
best_run, fitted_model = run.get_output()
print(best_run)
print('--------')
print(fitted_model)

Run(Experiment: SDK_local_porto_seguro_driver_pred,
Id: AutoML_51a190e0-5f8a-4742-b6ab-1db4153d3896_13,
Type: None,
Status: Completed)
--------
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667]))])


## Make Predictions and calculate metrics

### Prep Test Data: Separate x_test set (feature columns) from y_test (label column) 

In [26]:
import pandas as pd

x_test_df = test_df.copy()

if 'target' in x_test_df.columns:
    y_test_df = x_test_df.pop('target')

print(test_df.shape)
print(x_test_df.shape)
print(y_test_df.shape)


(59522, 59)
(59522, 58)
(59522,)


In [27]:
y_test_df.describe()

count   59522.00
mean        0.04
std         0.19
min         0.00
25%         0.00
50%         0.00
75%         0.00
max         1.00
Name: target, dtype: float64

### Make predictions in bulk

In [28]:
# Try the best model making predictions with the test dataset
y_predictions = fitted_model.predict(x_test_df)

print('30 predictions: ')
print(y_predictions[:30])

30 predictions: 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Get all the predictions' probabilities needed to calculate ROC AUC

In [29]:
class_probabilities = fitted_model.predict_proba(x_test_df)

print(class_probabilities.shape)

print('Some class probabilities...: ')
print(class_probabilities[:3])
print('')
print('Probabilities for class 1:')
print(class_probabilities[:,1])
print('')
print('Probabilities for class 0:')
print(class_probabilities[:,0])

(59522, 2)
Some class probabilities...: 
[[0.92059493 0.07940507]
 [0.92070912 0.07929088]
 [0.93865784 0.06134217]]

Probabilities for class 1:
[0.07940507 0.07929088 0.06134217 ... 0.06309826 0.06782013 0.05778116]

Probabilities for class 0:
[0.92059493 0.92070912 0.93865784 ... 0.93690175 0.93217986 0.94221884]


## Evaluate Model

Evaluating performance is an essential task in machine learning. In this case, because this is a classification problem, the data scientist elected to use an AUC - ROC Curve. When we need to check or visualize the performance of the multi - class classification problem, we use AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance.

<img src="https://www.researchgate.net/profile/Oxana_Trifonova/publication/276079439/figure/fig2/AS:614187332034565@1523445079168/An-example-of-ROC-curves-with-good-AUC-09-and-satisfactory-AUC-065-parameters.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 12px; width: 320px; height: 239px;" />

### Calculate the ROC AUC with probabilities vs. the Test Dataset

In [30]:
print('ROC AUC *method 1*:')
fpr, tpr, thresholds = metrics.roc_curve(y_test_df, class_probabilities[:,1])
metrics.auc(fpr, tpr)


ROC AUC *method 1*:


0.6339811902533052

In [31]:
from sklearn.metrics import roc_auc_score

print('ROC AUC *method 2*:')
print(roc_auc_score(y_test_df, class_probabilities[:,1]))

print('ROC AUC Weighted:')
print(roc_auc_score(y_test_df, class_probabilities[:,1], average='weighted'))


ROC AUC *method 2*:
0.6339811902533052
ROC AUC Weighted:
0.6339811902533052


### Calculate the Accuracy with predictions vs. the Test Dataset

In [32]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
print(accuracy_score(y_test_df, y_predictions))


Accuracy:
0.9634252881287592


## Download model pickle file from the run

In [33]:
# Download the model .pkl file to local (Using the 'run' object)
best_run.download_file('outputs/model.pkl')

### Load model in memory from .pkl file

In [34]:
# Load the model into memory
import joblib
fitted_model = joblib.load('model.pkl')
print(fitted_model)

Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667]))])
